<a href="https://colab.research.google.com/github/nabin-nath/major-project-2/blob/main/Synthesised_RandomSearchTune_e_consume_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 85.2 MB/s eta 0:00:00


In [4]:
#imports
import pandas as pd
import numpy as np

df0 = pd.read_csv('/content/drive/MyDrive/Major Project - II/0_interpolate_n.csv')
df1 = pd.read_csv('/content/drive/MyDrive/Major Project - II/1_interpolate_n.csv')

<ipython-input-4-19ce6e226e03>:5: DtypeWarning: Columns (0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263

In [5]:
df0 = df0.iloc[:,2:]
df1 = df1.iloc[:3579,2:]
df0 = df0.drop(df0.index[0])
df1 = df1.drop(df1.index[0])

df0 = df0.fillna(0)
df0 = df0.astype('float64')
df1 = df1.fillna(0)
df1 = df1.astype('float64')

from sklearn.utils import resample
import pandas as pd
import numpy as np

# Randomly select the rows
n_rows=len(df0)-len(df1)
df2 = df0.sample(n=n_rows)

# Define a random multiplication factor between 0 and 0.8
mult_factor = np.random.uniform(low=0, high=0.8, size=(n_rows, df0.shape[1]))

# Multiply each column of the selected rows with the multiplication factor
df_modified = df2 * mult_factor

# Add the modified rows to a new dataframe
df1 = df1.append(df_modified)
# df1 = resample(df1,
#              replace=True,
#              n_samples=len(df0),
#              random_state=42)
             
df = pd.concat([df0, df1], ignore_index=True)

<ipython-input-5-74c6f70fd3cb>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(df_modified)


In [7]:
print(len(df1), len(df0))

36679 36679


In [8]:
# dataset shuffling
df = df.sample(frac=1).reset_index(drop=True)

# test and train dataset preparation
trainX, trainy = df.iloc[:60000, 1:], df.iloc[:60000, :1]
testX, testy = df.iloc[60001:, 1:], df.iloc[60001:, :1]

# print(trainX.shape, trainy.shape, testX.shape, testy.shape)

In [9]:
#input scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(trainX)
scaled_train = scaler.transform(trainX)
scaled_test = scaler.transform(testX)

In [10]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras
# from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras_tuner.tuners import RandomSearch

In [11]:
import keras_tuner

In [12]:
class EeHyperModel(keras_tuner.HyperModel):
    def build(self, hp):
      n_timesteps, n_features, n_outputs = trainX.shape[1], 1, trainy.shape[1]
      model = Sequential()
      model.add(LSTM(hp.Int('units_LSTM', 80, 220, step=30), input_shape=(n_timesteps,n_features)))
      model.add(Dropout(hp.Choice('dropout', [0.2, 0.5])))

      for i in range(hp.Int('layers', 1, 3)):
        model.add(Dense(
          units=hp.Int('units_' + str(i), 30, 200, step=30),
          activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid'])    
        ))

      model.add(Dense(n_outputs, activation='sigmoid'))
      model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(
          hp.Choice('learning_rate',
                    values=[1e-2, 1e-4])),
                    metrics=['accuracy'])

      return model
    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Int("batch_size", 32, 264, step=32),
            **kwargs,
        )

In [13]:
tuner = RandomSearch(
    EeHyperModel(),
    objective='val_accuracy',
    max_trials=8,
    executions_per_trial=1,
    directory='my_dir',
    project_name='rnn_lstm'
)

In [14]:
tuner.search_space_summary()

Search space summary
Default search space size: 6
units_LSTM (Int)
{'default': None, 'conditions': [], 'min_value': 80, 'max_value': 220, 'step': 30, 'sampling': 'linear'}
dropout (Choice)
{'default': 0.2, 'conditions': [], 'values': [0.2, 0.5], 'ordered': True}
layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 30, 'max_value': 200, 'step': 30, 'sampling': 'linear'}
act_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.0001], 'ordered': True}


In [15]:
# Will stop training if the "val_loss" hasn't improved in 3 epochs.
tuner.search(trainX, trainy, epochs=2,
             validation_data=(testX,testy), 
             callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=3)])

Trial 8 Complete [00h 02m 25s]
val_accuracy: 0.9520101547241211

Best val_accuracy So Far: 0.9520101547241211
Total elapsed time: 00h 13m 54s


In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/rnn_lstm
Showing 10 best trials
Trial summary
Hyperparameters:
units_LSTM: 80
dropout: 0.2
layers: 1
units_0: 180
act_0: sigmoid
learning_rate: 0.01
units_1: 30
act_1: relu
units_2: 150
act_2: sigmoid
batch_size: 256
Score: 0.7092161178588867
Trial summary
Hyperparameters:
units_LSTM: 110
dropout: 0.2
layers: 3
units_0: 120
act_0: relu
learning_rate: 0.01
units_1: 60
act_1: relu
units_2: 150
act_2: relu
batch_size: 160
Score: 0.6788200736045837
Trial summary
Hyperparameters:
units_LSTM: 140
dropout: 0.2
layers: 3
units_0: 60
act_0: relu
learning_rate: 0.0001
units_1: 90
act_1: relu
units_2: 120
act_2: relu
batch_size: 192
Score: 0.6647450923919678
Trial summary
Hyperparameters:
units_LSTM: 200
dropout: 0.5
layers: 3
units_0: 150
act_0: sigmoid
learning_rate: 0.0001
units_1: 30
act_1: relu
units_2: 30
act_2: relu
batch_size: 32
Score: 0.6613011956214905
Trial summary
Hyperparameters:
units_LSTM: 140
dropout: 0.5
layers: 2
units_0: 90
act_0: relu
learnin